# Esercizio 3 - Hanks

- Il verbo scelto è **Play**
- Si estrae dal corpus **Brown** le istanze che contengono il verbo **Play**
- Si effettua parsing e disambiguazione delle frasi estratte
- Si usano i super sensi di WordNet sugli argomenti (subj e obj nel caso di 2 argomenti) del verbo scelto
- Si aggregano i risultati, si calcolano le frequenze, e si stampano i cluster semantici ottenuti

In [5]:
import nltk
from nltk.corpus import brown
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
import spacy

### Estrai le frasi che contengono il verbo Play dal corpus Brown

In [85]:
nlp = spacy.load('en_core_web_sm')
# verb = ['study', 'studies', 'studied']
verb = ['be', 'is', 'was']
sentences = [sent for sent in brown.sents() if verb[0] in sent or verb[1] in sent or verb[2] in sent]
print(len(sentences))
for i in range(10):
    print(' '.join(sentences[i]))

20932
The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted .
The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. .
`` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' .
However , the jury said it believes `` these two offices should be combined to achieve greater efficiency and reduce the cost of administration '' .
The City Purchasing Department , the jury said , `` is lacking in experienced clerical personnel as a result of city personnel policies '' .
Implementation of Georgia's automobile title law was a

### Parsing delle frasi estratte per trovare il soggetto e l'oggetto del verbo Play

Per ogni frase estratta dal corpus cerco il soggetto e l'oggetto del verbo **Play** e li salvo in una lista di tuple.  
Per farlo uso la libreria **spaCy** che mi permette di cercare il soggetto e l'oggetto di un verbo in una frase tramite `token.dep_ == 'nsubj'` e `token.dep_ == 'dobj'`.  
Tramite `token.head.text in verb` posso inoltre cercare se il soggetto e l'oggetto trovati sono effettivamente legati al verbo **Play**.  
Infine con `token.dep_ == 'ROOT'` posso sapere se il verbo **Play** è la radice della frase, senza questo controllo la ricerca del soggetto e dell'oggetto potrebbe portare a risultati non corretti, tipo pronomi o quant'altro.  
Mi sono accorto di ciò interpretando i risultati ottenuti.

In [86]:
# Parse the sentences using Spacy to extract the subject and object of the verb love
def get_subject_object(sentence):
    len = 0
    triples = []
    for sent in sentences:
        sub = ''
        obj = ''
        doc = nlp(' '.join(sent))
        isRoot = False
        for token in doc:
            if token.dep_ == 'nsubj' and token.head.text in verb:
                sub = token.text
            if token.dep_ == 'dobj' and token.head.text in verb:
                obj = token.text
            if token.dep_ == 'ROOT' and token.text in verb:
                isRoot = True
        if sub != '' and obj != '' and isRoot:
            len += 1
            triples.append((sub, obj, sent))
    return triples

# print(get_subject_object(sentences)[0:3])

### Disambiguazione delle coppie soggetto-oggetto

Per ogni coppia soggetto-oggetto estratta dal corpus, cerco il super-senso di WordNet che meglio descrive il soggetto e l'oggetto.

In [87]:
# Get the super senses of the subject and object
def get_super_sense(triples):
    super_senses = []
    for triple in triples:
        sub = triple[0]
        obj = triple[1]
        sent = triple[2]
        sub_sense = lesk(sent, sub)
        obj_sense = lesk(sent, obj)
        super_senses.append((sub_sense.lexname() if sub_sense else 'noun.person', obj_sense.lexname() if obj_sense else 'noun.person'))
    return super_senses

# print(get_super_sense(get_subject_object(sentences)))

### Aggregazione dei risultati e stampa dei cluster

In [88]:
# Aggregate the super senses
def aggregate_super_sense(super_senses):
    super_sense_dict = {}
    for super_sense in super_senses:
        if super_sense in super_sense_dict:
            super_sense_dict[super_sense] += 1
        else:
            super_sense_dict[super_sense] = 1
    super_sense_dict = {k: v for k, v in sorted(super_sense_dict.items(), key=lambda item: item[1], reverse=True)}
    return super_sense_dict

result = aggregate_super_sense(get_super_sense(get_subject_object(sentences)))
total = sum(result.values())
print(f'Total pairs: {len(result)}')
for key, value in result.items():
    print(f'{key}: {value/total*100:.2f}%')